# Load the data

In [1]:
from TF_StateFarm_Loader import *
import numpy as np

NUM_CLASSES = 10 # number of classes that we need
IMAGE_SIZE = 224 # reduce to this size to fit the VGG network
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE
LEARNING_RATE = 1e-4
DROPOUT = 0.5
BATCH_SIZE = 50

In [2]:
train_data, train_target = Load_Statefarm_Train()
num_examples = train_data.shape[0] # we have 22424 samples

read train images


In [3]:
rounds = np.floor(num_examples/BATCH_SIZE)
rounds

448.0

In [4]:
import tensorflow as tf

In [5]:
sess = tf.InteractiveSession()

In [6]:
x = tf.placeholder(tf.float32, shape=[BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3])
y_ = tf.placeholder(tf.float32, shape=[BATCH_SIZE])
keep_prob = tf.placeholder(tf.float32)

In [7]:
tf.__version__

'0.9.0'

In [8]:
# layers
def get_bias(shape):
    return tf.truncated_normal(shape, stddev=0.1, dtype=tf.float32)
        #return tf.constant([name][1], name="biases")

def conv_layer(bottom, name, filters, channels):
        with tf.variable_scope(name):
            filt = tf.truncated_normal([3,3,channels,filters], dtype=tf.float32)
            conv = tf.nn.conv2d(bottom, filt, [1, 1, 1, 1], padding='SAME')
            conv_biases = tf.truncated_normal([filters], stddev=0.1,  dtype=tf.float32)
            bias = tf.nn.bias_add(conv, conv_biases)

            relu = tf.nn.relu(bias)
            return relu

def max_pool(bottom, name):
        return tf.nn.max_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], 
                              padding='SAME', name=name)
        
conv1_1 = conv_layer(x, "conv1_1", 64, 3)
conv1_2 = conv_layer(conv1_1, "conv1_2", 64, 64)
pool1 = max_pool(conv1_2, "pool_1")

conv2_1 = conv_layer(pool1, "conv2_1", 128, 64)
conv2_2 = conv_layer(conv2_1, "conv2_1", 128, 128)
pool2 = max_pool(conv2_2, "pool_2")

conv3_1 = conv_layer(pool2, "conv3_1", 256, 128)
conv3_2 = conv_layer(conv3_1, "conv3_2", 256, 256)
conv3_3 = conv_layer(conv3_2, "conv3_3", 256, 256)
pool3 = max_pool(conv3_3, "pool_3")

conv4_1 = conv_layer(pool3, "conv4_1", 512, 256)
conv4_2 = conv_layer(conv4_1, "conv4_2", 512, 512)
conv4_3 = conv_layer(conv4_2, "conv4_3", 512, 512)
pool4 = max_pool(conv4_3, "pool_4")

shape = int(np.prod(pool4.get_shape()[1:]))
fc1w = tf.Variable(tf.truncated_normal([shape, 10],dtype=tf.float32,stddev=1e-1), name='weights')
fc1b = tf.Variable(tf.constant(1.0, shape=[10], dtype=tf.float32),
                                 trainable=True, name='biases')
pool5_flat = tf.reshape(pool4, [-1, shape])
fc1l = tf.nn.bias_add(tf.matmul(pool5_flat, fc1w), fc1b)
fc1 = tf.nn.relu(fc1l)

In [9]:
# need to declare all variables first
sess.run(tf.initialize_all_variables())

In [10]:
probs = tf.nn.softmax(fc1)

In [ ]:
# Loop through and make the training batches

for i in range(0, int(rounds) ):

    start = i * BATCH_SIZE
    
    if (i < rounds-1) :    
        end = (i + 1) * BATCH_SIZE
    else:
        end = num_examples
    
    Train_Batch_x = train_data[start:end]
    Train_Batch_y = train_target[start:end]
    print 'Start: %d, End: %d' % (start, end)
    print Train_Batch_x.shape
    print Train_Batch_y.shape
    Train_Batch_x = Train_Batch_x.astype('float')
    results = sess.run(probs, feed_dict={x: Train_Batch_x, y_:Train_Batch_y})
    # TODO TF on Batch Code
        

Start: 0, End: 50
(50, 224, 224, 3)
(50,)
Start: 50, End: 100
(50, 224, 224, 3)
(50,)
Start: 100, End: 150
(50, 224, 224, 3)
(50,)


### old testing code

In [ ]:
from NN_Components import *

In [ ]:
train_data.shape
train_target.shape

In [ ]:
train_data_s = train_data.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
train_target_s = to_categorical(train_target, NUM_CLASSES)

In [3]:
import NN_Components
#reload(NN_Components)
%autoreload 2

In [ ]:
train_data_s.shape

In [ ]:
Batches = train_data_s.shape[0] / BATCH_SIZE
Batches

In [ ]:
train_target_s.shape

In [13]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [14]:
x = tf.placeholder('float', shape=[BATCH_SIZE, image_width, image_width, 3])
y_ = tf.placeholder('float', shape=[BATCH_SIZE, NUM_CLASSES])
keep_prob = tf.placeholder('float')

# do I initiatlise here?

In [ ]:
#def conv_net(x, weights, biases, dropout):
    
W1 = weight_variable(shape = [3, 3, 3, 64])
b1 = bias_variable(shape = [64])

a1 = tf.nn.conv2d(x, W1, strides=[1, 1, 1, 1], padding='SAME')
a2 = tf.nn.relu(a1 + b1)

W2 = weight_variable(shape = [3, 3, 64, 64])
b2 = bias_variable(shape = [64])

a3 = tf.nn.conv2d(a2, W2, strides=[1, 1, 1, 1], padding='SAME')
a4 = tf.nn.relu(a3 + b2)

pooled = max_pool_2x2(a4,colours = 1)

# dense connect
W_fc1 = weight_variable([112 * 112 * 64, 4096])
b_fc1 = bias_variable([4096])

h_pool2_flat = tf.reshape(pooled, [-1, 112*112*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# drop out
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([4096, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())


In [ ]:
for i in range(20000):
  batch = mnist.train.next_batch(50) # need to fix
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

In [5]:
# model
#SHAPES = [batch, ]
def inference(images):
    """
    Args:
        images: 4D tensor of [batch_size, IMAGE_SIZE, IMAGE_SIZE, 3] size.
    Returns:
        Logits
    Raises:
        
    """
    #inputshape = [1]
    with tf.variable_scope('conv1') as scope:
        
        # VGG zero pads first
        
        weights = weight_variable([5, 5, 3, 64]) # 5x5 filter across 3 colour channels 64 output
        conv = tf.nn.conv2d(images, weights, strides = [1,2,2,1], padding = 'SAME')
        biases = bias_variable([64])
        bias = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(bias, name = scope.name)
        # logging here
        
        pool1 = tf.nn.max_pool(conv1, ksize=[1,3,3,1], strides = [1,2,2,1], 
                               padding='SAME', name='pool1')
        
        norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                         name = 'norm1')

    return norm1

In [13]:
# testing framework
import tensorflow as tf

In [16]:
matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2.],[2.]])
product = tf.matmul(matrix1, matrix2)

In [18]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [19]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [20]:
sess.run(tf.initialize_all_variables())

In [21]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

In [22]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [23]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)